<a href="https://colab.research.google.com/github/duynht/Greedy_InfoMax/blob/master/Gradient_isolated_GNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# README
This notebook houses preliminary experiments for applying gradient-isolated training to Graph Neural Networks

# Keep Colab running in the background
Set a JavaScript interval to click on the connect button every 60 seconds. Open developer-settings (in your web-browser) with Ctrl+Shift+I then click on console tab and paste this to the console prompt. (for Mac press Option+Command+I)



```
<!-- function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000); -->
```
Little did I know Colab changed their selector id with this fancy update so we'll use the now code then. This time we have proper start and stop function. LOLOLOLOLOL

```
var startClickConnect = function startClickConnect(){
    var clickConnect = function clickConnect(){
        console.log("Connnect Clicked - Start");
        document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
        console.log("Connnect Clicked - End"); 
    };

    var intervalId = setInterval(clickConnect, 60000);

    var stopClickConnectHandler = function stopClickConnect() {
        console.log("Connnect Clicked Stopped - Start");
        clearInterval(intervalId);
        console.log("Connnect Clicked Stopped - End");
    };

    return stopClickConnectHandler;
};

var stopClickConnect = startClickConnect();
```


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
!git clone https://github.com/duynht/Greedy_InfoMax.git
%cd /content/Greedy_InfoMax

Cloning into 'Greedy_InfoMax'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 360 (delta 5), reused 10 (delta 2), pack-reused 345
Receiving objects: 100% (360/360), 35.27 MiB | 50.44 MiB/s, done.
Resolving deltas: 100% (197/197), done.
/content/Greedy_InfoMax


# Graph package installation 

In [9]:
!pip install -r requirements.txt

     |████████████████████████████████| 753.4MB 21kB/s 
ERROR: torchaudio 0.5.0 has requirement torch==1.5.0, but you'll have torch 1.4.0 which is incompatible.
  Found existing installation: torch 1.5.0
    Uninstalling torch-1.5.0:
      Successfully uninstalled torch-1.5.0


# Import

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision.models as models
import torchvision.datasets as datasets
import torchvision.transforms as transforms

import torch_geometric.nn as pyg_nn

import torch_geometric.utils as pyg_utils

import networkx as nx
import numpy as npnpinpi
import torch
import torch.optim as optim

from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.data import InMemoryDataset
from torch_geometric.data import DataLoader
from torch_geometric.data import Data

import torch_geometric.transforms as T

from tensorboardX import SummaryWriter
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

import os.path as osp
from PIL import Image
import torch_geometric.transforms as T

from tensorboardX import SummaryWriter
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

import os.path as osp
from PIL import Image

In [5]:
from GreedyInfoMax.vision.data import get_dataloader
from GreedyInfoMax.vision.arg_parser import arg_parser
from GreedyInfoMax.vision.models import load_vision_model, SmallModel
from GreedyInfoMax.utils import logger, utils

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
import pdb
# breakpoint() # not working for python 3.6

In [0]:
import sys
if '-f' in sys.argv:
  sys.argv.remove('-f')

# Create a Custom Graph Dataset from STL10
Each image is divided into a $2\times2$ grid. Each patch of the grid is a node of the graph.


In [0]:
class GraphSTL10(InMemoryDataset):
  def __init__(self, root, split):
    # !python -m GreedyInfoMax.vision.modded_downstream_classification \
    # --model_path "/content/drive/My Drive/Greedy_InfoMax/logs/modded_vision_experiment_module_all" \
    # --model_num 5 \
    # --download_dataset
    self.opt = arg_parser.parse_args()
    self.opt.download_dataset=True
    self.opt.model_path = "/content/drive/My Drive/pretrainted-resnet50-infomax"
    self.opt.model_num = 5    
    
    self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    self.context_models = []
    for pid in range(4):
      context_model, _ = load_vision_model.load_model_and_optimizer(
                self.opt, reload_model=True, calc_loss=False, patch_idx=pid)
      context_model.module.switch_calc_loss(False)
      for param in context_model.parameters():
        param.requires_grad = False
      context_model.eval()
      context_model.to(self.device)
      self.context_models.append(context_model)
      
    self.dataloader = None

    _, _, trainloader, _, testloader, _ = get_dataloader.get_dataloader(self.opt)
    self.split = split
    if (split == 'train'):
      # self.dataset = datasets.STL10(root='/tmp/stl10_train', split='train', download=True)
      self.dataloader = trainloader
    if (split == 'test'):
      # self.dataset = datasets.STL10(root='/tmp/stl10_test', split='test', download=True)
       self.dataloader = testloader
    
    super(GraphSTL10, self).__init__(root)
    # self.num_classes = 10
    self.data, self.slices = torch.load(self.processed_paths[0])

  @property
  def raw_file_names(self):
    return []

  @property
  def processed_file_names(self):
    if (self.split == 'train'):
      return ['graphstl10_train.pt']
    if (self.split == 'test'):
      return ['graphstl10_test.pt']
    
    return []

  def download(self):
    pass

  def process(self):
    
    """Pytorch ResNet50
    # def crop(image,pc_height,pc_width):
    #   im_width, im_height = image.size
    #   for i in range(im_height//pc_height):
    #     for j in range(im_width//pc_width):
    #       box = (j*pc_width, i*pc_height, (j+1)*pc_width, (i+1)*pc_height)
    #       yield image.crop(box)

    # data_list = []

    # preprocess = transforms.Compose([transforms.Resize(256),
    #                                     transforms.CenterCrop(224),
    #                                     transforms.ToTensor(),
    #                                     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

    # resnet50 = models.resnet50(pretrained=True)
    # resnet50.fc = nn.Identity()

    # for param in resnet50.parameters():
    #   param.requires_grad = False

    # device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # resnet50.to(device)

    # source_nodes = [i for i in range(0,4) for j in range(0,4)]
    # target_nodes = [j for i in range(0,4) for j in range(0,4)]
    # edge_index = torch.tensor([source_nodes, target_nodes],dtype = torch.long)

    # for imid, (image, label) in enumerate(self.dataset):
    #   # neighbors = np.arange(imid, imid+4)
    #   # mask = [1]*len(neighbors)
    #   im_height, im_width = image.size
    #   pc_height, pc_width = im_height//2, im_width//2
    #   node_features = []
    #   for pid,piece in enumerate(crop(image, pc_height, pc_width)):
    #     patch = Image.new('RGB', (pc_width, pc_height), 255)
    #     patch.paste(piece)
    #     patch = preprocess(patch)
    #     patch = patch.view(1,*patch.shape)
    #     patch = patch.to(device)
    #     patch = resnet50.forward(patch).to(torch.device('cpu'))
    #     node_features.append(torch.tensor(patch))
    """

    source_nodes = [i for i in range(0,4) for j in range(0,4)]
    target_nodes = [j for i in range(0,4) for j in range(0,4)]
    edge_index = torch.tensor([source_nodes, target_nodes],dtype = torch.long)

    data_list = []

    for bid, (imgs, labels) in enumerate(self.dataloader):
      batch_size, num_channels, img_height, img_width = imgs.shape
      if not(batch_size == self.opt.batch_size_multiGPU):
        continue
      patches = []
      node_features = []
      patches.append(imgs[:,:, :img_height//2, :img_width//2])
      patches.append(imgs[:,:, :img_height//2, img_width//2:])
      patches.append(imgs[:,:, img_height//2:, :img_width//2])
      patches.append(imgs[:,:, img_height//2:, img_width//2:])
      for pid in range(4):
        torch.cuda.empty_cache()
        patch = patches[pid].to(self.device)
        patch = self.context_models[pid](patch,labels)[2]
        patch = patch.to(torch.device('cpu'))
        node_features.append(patch)

      node_features = torch.stack(node_features)
      node_features = node_features.transpose(0,1)
      node_features = node_features.view(*node_features.shape[:-3],-1)
      # pdb.set_trace()
      labels = labels.to(torch.device('cpu'))
      for gid in range(batch_size):  
        data = Data(x=node_features[gid], edge_index=edge_index.clone(), y=labels[gid].unsqueeze(0))
        data_list.append(data)
      # pdb.set_trace()
    
    data, slices = self.collate(data_list)
    torch.save((data, slices), self.processed_paths[0])
  

# Define custom MessagePassing [WIP]


In [0]:
class CustomConv(pyg_nn.MessagePassing):
  def __init__(self, in_channels, out_channels):
    super(CustomConv, self).__init__(aggr='add') #'add' aggregation
    self.lin = nn.Linear(in_channels, out_channels)
    self.lin_self = nn.Linear(in_channels, out_channels)

  def forward(self, x, edge_index):
    # x has shape [N, in_channels]
    # edge_index has shape [2, E]

    # Transform node feature matrix
    self_x = self.lin_self(x)
    # x = self.lin(x)

    return self_x + self.propagate(edge_index, size=(x.size(0), x.size(0)), x=self.lin(x))
  
  def message(self, x_i, x_j, edge_index, size):
    # Compute messages
    # x_j has shape [E, out_channels]
    # TODO:
    row, col = edge_index
    deg = pyg_utils.degree(row, size[0], dtype=x_j.dtype)
    deg_inv_sqrt = deg.pow(-0.5)
    norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

    return x_j

  def update(self, aggr_out):
    # aggr_out has shape [N, out_channels]
    return aggr_out

# Define the Graph Neural Network

In [0]:
class VisionGNN(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim):
    super(VisionGNN, self).__init__()
        
    self.dropout = 0.25
    self.num_layers = 2
    self.hidden = [input_dim, 512, hidden_dim]
    # self.resnet = models.resnet50(pretrained=True)
    # self.resnet.fc = nn.Identity()

    self.convs = nn.ModuleList()
    self.lns = nn.ModuleList()

    for l in range(self.num_layers):
      self.convs.append(self.build_conv_model(self.hidden[l], self.hidden[l+1]))
      if (l + 1 < self.num_layers):
        self.lns.append(nn.LayerNorm(self.hidden[l+1]))

    # post-message-passing
    self.post_mp = nn.Sequential(
        nn.Linear(hidden_dim, hidden_dim), nn.Dropout(0.25),
        nn.Linear(hidden_dim, output_dim))

  def build_conv_model(self, input_dim, hidden_dim):
      # return CustomConv(input_dim, hidden_dim)
      return pyg_nn.GCNConv(input_dim, hidden_dim)
      
  def forward(self, data):
    x, edge_index = data.x, data.edge_index
    if data.num_node_features == 0:
      x = torch.ones(data.num_nodes, 1)

    for i in range(self.num_layers):
      x = self.convs[i](x, edge_index)
      emb = x
      x = F.relu(x)
      x = F.dropout(x, p=self.dropout, training = self.training)
      if not i == self.num_layers - 1:
        x = self.lns[i](x)

    # x = pyg_nn.global_mean_pool(x, data.batch)
    x = pyg_nn.global_add_pool(x, data.batch)

    x = self.post_mp(x)

    return emb, F.log_softmax(x, dim=1)

  def loss(self, pred, label):
    # Negative log-likelihood
    return F.nll_loss(pred, label)

In [0]:
def train(model, optimizer):
  trainloader = DataLoader(stl10_train, batch_size=64, shuffle=True)
  testloader = DataLoader(stl10_test, batch_size=64, shuffle=True)
  
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model.to(device)

  epochs = 200
  for e in range(epochs):
    running_loss = 0
    train_acc = 0
    model.train()
    for batch in trainloader:
      optimizer.zero_grad()
      batch = batch.to(device)
      emb, logits = model(batch)
      labels = batch.y
      loss = model.loss(logits, labels)
      pred = logits.argmax(dim=1) 

      train_acc += pred.eq(labels).sum().item()
      running_loss += loss.item()

      loss.backward()
      optimizer.step()

    train_acc /= len(trainloader.dataset)
    running_loss /= len(trainloader.dataset)
    if e % 5 == 0:
      test_acc = test(model, testloader)
      print("Epoch {}/{}. Loss: {:.4f}. Train accuracy: {:.4f}. Test accuracy: {:.4f}".format(e+1, epochs, running_loss, train_acc, test_acc))
     
        


In [0]:
def test(model, testloader):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  accuracy = 0
  model.eval()
  for batch in testloader:
    with torch.no_grad():
      batch = batch.to(device)
      emb, logits = model(batch)
      pred = logits.argmax(dim=1)
      labels = batch.y
    
    accuracy += pred.eq(labels).sum().item()

  accuracy /= len(testloader.dataset)
  return accuracy


In [0]:
torch.cuda.empty_cache()

In [0]:
# !rm -r /graphstl10/

In [42]:
stl10_train = GraphSTL10('/graphstl10/',split='train')
stl10_test = GraphSTL10('/graphstl10/',split='test')

Contrasting against  16  negative samples
ModuleList(
  (0): ModuleList(
    (0): ResNet_Encoder(
      (model): Sequential(
        (Conv1): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
        (layer 0): Sequential(
          (0): PreActBottleneckNoBN(
            (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
            (shortcut): Sequential(
              (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
            )
          )
          (1): PreActBottleneckNoBN(
            (conv1): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1))
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
          )
          (2): PreActBottleneckNoBN(
            (conv1): Conv2d(256, 64, kernel_si

In [43]:
model = VisionGNN(input_dim=stl10_train.num_node_features, hidden_dim=64, output_dim = 10)
optimizer = optim.Adam(params=model.parameters(), lr=0.003)
train(model, optimizer)

Epoch 1/200. Loss: 0.0309. Train accuracy: 0.2340. Test accuracy: 0.2966
Epoch 6/200. Loss: 0.0202. Train accuracy: 0.5032. Test accuracy: 0.4276
Epoch 11/200. Loss: 0.0143. Train accuracy: 0.6536. Test accuracy: 0.4336
Epoch 16/200. Loss: 0.0102. Train accuracy: 0.7610. Test accuracy: 0.4442
Epoch 21/200. Loss: 0.0060. Train accuracy: 0.8660. Test accuracy: 0.4444
Epoch 26/200. Loss: 0.0052. Train accuracy: 0.8840. Test accuracy: 0.4293
Epoch 31/200. Loss: 0.0035. Train accuracy: 0.9263. Test accuracy: 0.4412
Epoch 36/200. Loss: 0.0029. Train accuracy: 0.9399. Test accuracy: 0.4461
Epoch 41/200. Loss: 0.0027. Train accuracy: 0.9443. Test accuracy: 0.4475
Epoch 46/200. Loss: 0.0023. Train accuracy: 0.9567. Test accuracy: 0.4562
Epoch 51/200. Loss: 0.0019. Train accuracy: 0.9651. Test accuracy: 0.4650
Epoch 56/200. Loss: 0.0024. Train accuracy: 0.9513. Test accuracy: 0.4432
Epoch 61/200. Loss: 0.0016. Train accuracy: 0.9655. Test accuracy: 0.4510
Epoch 66/200. Loss: 0.0016. Train accura

In [44]:
%debug

> /usr/lib/python3.6/bdb.py(70)dispatch_line()
     68         if self.stop_here(frame) or self.break_here(frame):
     69             self.user_line(frame)
---> 70             if self.quitting: raise BdbQuit
     71         return self.trace_dispatch
     72 

ipdb> quit
